This code file includes the full process to produce the comprehensive output dataset, which can be found easily at the link:

https://docs.google.com/spreadsheets/d/1JCpMYvMVlr_jyRRL7RkdoAfu8AFzugKB3-DxloH4AuU/edit?gid=1866890265#gid=1866890265

Overview of process:

Regex Algorithm → Extract Location, State, Route, Time, Altitude, Authority Advised
Regex Algorithm → Extract 3 letter FAA code representing information of NAVAID/airport details.
Web Scraping Algorithm → Extract 3 letter codes + longitude/latitude information of certain edge cases.
Merge on a DataFrame containing longitude, latitude, altitude of all airports, and get the latitude and longitude for majority cases where we don’t need web scraping.
Extracting the 3D coordinates for the airport based on longitude, latitude, altitude.

This comprehensive data processing step allows me to go from the above messy dataset, containing all information in unparsable string format, into this dataset structure, which can easily be graphed and analyzed by anyone with basic Python programming knowledge. One of the most important key deliverables for me here is developing a very well documented Python notebook that can walk any other Berkeley student through the process of developing the below DataFrame from the FAA output. My supervisor requested this deliverable out of me, since she has a lot of students studying this topic and creating it would be helpful for her lab. 
	
Most sightings from the dataset are any UAS sightings obtained from pilots or residents. Something that may cause huge data bias issues for me is that the FAA has no way to verify whether these sightings are legitimate drones, but just keeps them as a sighting within their public records as reported. There are very few cases in which drone intrusions lead to evasive actions, but this number has been increasing since 2014. Finally, there is a lot of criticism towards the FAA due to its poor management of its data records, which I can see as I analyze them, which is a data quality issue I have to deal with. However, the data is government-obtained, which means it is a trustable source, just a source that might be lacking in 100% quality.


In [1]:
import pandas as pd
import pandas as pd
import os
import numpy as np
import re
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import zipfile
import time

In [2]:
dataset_folder = "drone-intrusion/all_datasets"
dataset_folder

'drone-intrusion/all_datasets'

In [ ]:
import pandas as pd
import glob
import os

dataset_folder = "all_datasets"

xlsx_files = glob.glob(os.path.join(dataset_folder, "*.xlsx"))

# concatenate all files into one
df_all = pd.concat([pd.read_excel(file) for file in xlsx_files], ignore_index=True)

print(df_all.shape)
df_all.head()

(8830, 16387)


,Day of Sighting,State,City,Summary,Date,Column1,Column2,Column3,Column4,Column5,...,Column16373,Column16374,Column16375,Column16376,Column16377,Column16378,Column16379,Column16380,Date of Sighitng,Date of Sighting
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [8]:
df_all_condensed = df_all[['Day of Sighting',
 'State',
 'City',
 'Summary',
 'Date']]

df_all_condensed

,Day of Sighting,State,City,Summary,Date
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",NaT
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",NaT
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",NaT
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT
...,...,...,...,...,...
8825,NaN,FLORIDA,WEST PALM BEACH,"PRELIM INFO FROM FAA OPS: WEST PALM BEACH, FL/...",NaT
8826,NaN,FLORIDA,FORT MYERS,"PRELIM INFO FROM FAA OPS: FORT MYERS, FL/UAS I...",NaT
8827,NaN,FLORIDA,NAPLES,_x000D_\nSummary: A/C WAS SOUTHBOUND TO APF AT...,NaT
8828,NaN,HAWAII,HONOLULU,"PRELIM INFO FROM FAA OPS: HONOLULU, HI/UAS INC...",NaT


In [9]:
zip_file_path_airport = "all-airport-data.xlsx.zip"
xlsx_file_name_airport = "all-airport-data.xlsx"

with zipfile.ZipFile(zip_file_path_airport) as z:
    with z.open(xlsx_file_name_airport) as f:
        airport_codes_2 = pd.read_excel(f)

/Users/vishalivallioor/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
''''
Line of code to extract all the 3 letter airport codes. 
'''

list_codes = list(airport_codes_2["Loc Id"])

df_all_condensed['code'] = None

for index_2, row in df_all_condensed.iterrows():
  x = row["Summary"]
  splitted_x = x.split()
  punc = '''!()-[]{};:'"\,<>.,/?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  index = 0
  for word in splitted_x:
      for letter in word:
        if letter in punc:
          word = word.replace(letter, "")
          splitted_x[index] = word
      index = index + 1
  filtered_codes = list(set([i for i in splitted_x if len(i) == 3 and i not in ["FAA", "UAS", "OPS", "THE"]]))
  for i in filtered_codes:
    if i in list_codes:
       df_all_condensed.at[index_2, 'code'] = i

df_all_condensed

/var/folders/97/l1_w165s1z38tw82rlydjzp00000gn/T/ipykernel_89088/2610744215.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_condensed['code'] = None


,Day of Sighting,State,City,Summary,Date,code
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",NaT,AGC
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",NaT,OPF
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,BFI
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",NaT,BFM
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,SEA
...,...,...,...,...,...,...
8825,NaN,FLORIDA,WEST PALM BEACH,"PRELIM INFO FROM FAA OPS: WEST PALM BEACH, FL/...",NaT,DFW
8826,NaN,FLORIDA,FORT MYERS,"PRELIM INFO FROM FAA OPS: FORT MYERS, FL/UAS I...",NaT,APF
8827,NaN,FLORIDA,NAPLES,_x000D_\nSummary: A/C WAS SOUTHBOUND TO APF AT...,NaT,APF
8828,NaN,HAWAII,HONOLULU,"PRELIM INFO FROM FAA OPS: HONOLULU, HI/UAS INC...",NaT,HNL


In [11]:
'''
All regex commands for drone information extraction.
'''

def extract_direction(summary):
    match = re.search(r'WHILE\s([A-Z]+\sBOUND)', summary)
    if match:
        return match.group(1)
    else:
        return np.nan

df_all_condensed['Direction'] = df_all_condensed['Summary'].apply(extract_direction)

pattern_time = r'(?<=\/)([0-9]{4}[A-Z])'
df_all_condensed['Time'] = df_all_condensed['Summary'].str.extract(pattern_time)

pattern_advised = r'(?<=\/)([A-Z ]+)(?= ADVISED)'
df_all_condensed['Authority Advised'] = df_all_condensed['Summary'].str.extract(pattern_advised)

pattern_route = r'(?<=, )([A-Z  ,()]+ - [A-Z  ,()]+)(?=,)'
df_all_condensed['Route'] = df_all_condensed['Summary'].str.extract(pattern_route)

pattern_location = r'(?<= )([0-9]+ [(W|ESE|E|N|NW|SW|S|SE|NE|NNE|NNW|WNW|WSW|NORTHWEST|ENE|SSE|SSW)]+ [A-Z() ]+)(?= AT| FEET|[.])'
df_all_condensed['Location'] = df_all_condensed['Summary'].str.extract(pattern_location)

pattern_altitude = r'(?<= )(AT [0-9,]+ FEET)(?= )'
df_all_condensed['Altitude'] = df_all_condensed['Summary'].str.extract(pattern_altitude)

df_all_condensed

/var/folders/97/l1_w165s1z38tw82rlydjzp00000gn/T/ipykernel_89088/3527091040.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_condensed['Direction'] = df_all_condensed['Summary'].apply(extract_direction)
/var/folders/97/l1_w165s1z38tw82rlydjzp00000gn/T/ipykernel_89088/3527091040.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_condensed['Time'] = df_all_condensed['Summary'].str.extract(pattern_time)
/var/folders/97/l1_w165s1z38tw82rlydjzp00000gn/T/ipykernel_89088/3527091040.py:18: Setti

,Day of Sighting,State,City,Summary,Date,code,Direction,Time,Authority Advised,Route,Location,Altitude
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",NaT,AGC,S BOUND,2358E,ALLEGHENY COUNTY ATCT,NaN,6 N AGC,"AT 1,600 FEET"
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",NaT,OPF,W BOUND,1205E,NaN,DAL - OPF,5 E OPF,"AT 1,500 FEET"
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,BFI,NNE BOUND,1210P,SEATTLE TRACON,HDN - BFI,15 WNW SEATTLE,"AT 3,000 FEET"
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",NaT,BFM,SE BOUND,1859C,MOBILE APCH,BFM - FMY,14 N SEMMES VOR,"AT 6,000 FEET"
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,SEA,W BOUND,1805P,SEATTLE TRACON,"KEFLAVIK, ICELAND (BIKF) - SEA",17 N SEA,"AT 7,000 FEET"
...,...,...,...,...,...,...,...,...,...,...,...,...
8825,NaN,FLORIDA,WEST PALM BEACH,"PRELIM INFO FROM FAA OPS: WEST PALM BEACH, FL/...",NaT,DFW,E BOUND,1800E,PALM BEACH TRACON,DFW - PBI,1 W PALM BEACH,AT 400 FEET
8826,NaN,FLORIDA,FORT MYERS,"PRELIM INFO FROM FAA OPS: FORT MYERS, FL/UAS I...",NaT,APF,S BOUND,1655E,FORT MYERS APCH,JVY - APF,8 SE FORT MYERS,"AT 7,000 FEET"
8827,NaN,FLORIDA,NAPLES,_x000D_\nSummary: A/C WAS SOUTHBOUND TO APF AT...,NaT,APF,NaN,NaN,NaN,NaN,NaN,NaN
8828,NaN,HAWAII,HONOLULU,"PRELIM INFO FROM FAA OPS: HONOLULU, HI/UAS INC...",NaT,HNL,NaN,1512L,NaN,"HNL - LAHAINA, HI (PHJH)",NaN,"AT 1,000 FEET"


In [12]:
works_df = df_all_condensed[~df_all_condensed["code"].isna()]
works_df

,Day of Sighting,State,City,Summary,Date,code,Direction,Time,Authority Advised,Route,Location,Altitude
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",NaT,AGC,S BOUND,2358E,ALLEGHENY COUNTY ATCT,NaN,6 N AGC,"AT 1,600 FEET"
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",NaT,OPF,W BOUND,1205E,NaN,DAL - OPF,5 E OPF,"AT 1,500 FEET"
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,BFI,NNE BOUND,1210P,SEATTLE TRACON,HDN - BFI,15 WNW SEATTLE,"AT 3,000 FEET"
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",NaT,BFM,SE BOUND,1859C,MOBILE APCH,BFM - FMY,14 N SEMMES VOR,"AT 6,000 FEET"
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,SEA,W BOUND,1805P,SEATTLE TRACON,"KEFLAVIK, ICELAND (BIKF) - SEA",17 N SEA,"AT 7,000 FEET"
...,...,...,...,...,...,...,...,...,...,...,...,...
8825,NaN,FLORIDA,WEST PALM BEACH,"PRELIM INFO FROM FAA OPS: WEST PALM BEACH, FL/...",NaT,DFW,E BOUND,1800E,PALM BEACH TRACON,DFW - PBI,1 W PALM BEACH,AT 400 FEET
8826,NaN,FLORIDA,FORT MYERS,"PRELIM INFO FROM FAA OPS: FORT MYERS, FL/UAS I...",NaT,APF,S BOUND,1655E,FORT MYERS APCH,JVY - APF,8 SE FORT MYERS,"AT 7,000 FEET"
8827,NaN,FLORIDA,NAPLES,_x000D_\nSummary: A/C WAS SOUTHBOUND TO APF AT...,NaT,APF,NaN,NaN,NaN,NaN,NaN,NaN
8828,NaN,HAWAII,HONOLULU,"PRELIM INFO FROM FAA OPS: HONOLULU, HI/UAS INC...",NaT,HNL,NaN,1512L,NaN,"HNL - LAHAINA, HI (PHJH)",NaN,"AT 1,000 FEET"


In [13]:
works_df.to_csv("big_dataset_works.csv")

In [14]:
df_all_condensed.to_csv("big_dataset_everything.csv")

In [16]:
airport_code_df = pd.read_csv("airport-codes.csv")
airport_code_df.dropna(axis = 0, subset = "iata_code")

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,icao_code,iata_code,gps_code,local_code,coordinates
204,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,NaN,UTK,03N,03N,"11.222219, 169.851429"
409,07FA,small_airport,Ocean Reef Club Airport,8.0,NaN,US,US-FL,Key Largo,NaN,OCA,07FA,07FA,"25.325399398804, -80.274803161621"
438,07TE,small_airport,Cuddihy Field,39.0,NaN,US,US-TX,Corpus Christi,NaN,CUX,07TE,07TE,"27.7211, -97.512802"
630,0CO2,small_airport,Crested Butte Airpark,8980.0,NaN,US,US-CO,Crested Butte,NaN,CSE,0CO2,0CO2,"38.851918, -106.928341"
885,0NM0,small_airport,Columbus Airport,4024.0,NaN,US,US-NM,Columbus,NaN,CUS,0NM0,0NM0,"31.823898, -107.629924"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82798,ZYTX,large_airport,Shenyang Taoxian International Airport,198.0,AS,CN,CN-21,"Hunnan, Shenyang",ZYTX,SHE,ZYTX,NaN,"41.639801, 123.483002"
82799,ZYXC,medium_airport,Xingcheng Air Base,30.0,AS,CN,CN-21,Huludao (Xingcheng),ZYXC,XEN,ZYXC,NaN,"40.580328, 120.700374"
82800,ZYYJ,medium_airport,Yanji Chaoyangchuan Airport,624.0,AS,CN,CN-22,Yanji,ZYYJ,YNJ,ZYYJ,NaN,"42.8828010559, 129.451004028"
82801,ZYYK,medium_airport,Yingkou Lanqi Airport,NaN,AS,CN,CN-21,Yingkou (Laobian),ZYYK,YKH,ZYYK,NaN,"40.542524, 122.3586"


In [18]:
merged_works_df = works_df.merge(airport_code_df, left_on = "code", right_on = "iata_code")
merged_works_df_usa = merged_works_df[merged_works_df["iso_country"] == "US"]
merged_works_df_usa

,Day of Sighting,State,City,Summary,Date,code,Direction,Time,Authority Advised,Route,...,elevation_ft,continent,iso_country,iso_region,municipality,icao_code,iata_code,gps_code,local_code,coordinates
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",NaT,AGC,S BOUND,2358E,ALLEGHENY COUNTY ATCT,NaN,...,1252.0,NaN,US,US-PA,Pittsburgh,KAGC,AGC,KAGC,AGC,"40.354401, -79.930199"
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",NaT,OPF,W BOUND,1205E,NaN,DAL - OPF,...,8.0,NaN,US,US-FL,Miami,KOPF,OPF,KOPF,OPF,"25.907, -80.278397"
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,BFI,NNE BOUND,1210P,SEATTLE TRACON,HDN - BFI,...,21.0,NaN,US,US-WA,Seattle,KBFI,BFI,KBFI,BFI,"47.529999, -122.302002"
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",NaT,BFM,SE BOUND,1859C,MOBILE APCH,BFM - FMY,...,26.0,NaN,US,US-AL,Mobile,KBFM,BFM,KBFM,BFM,"30.626800537100003, -88.06809997559999"
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",NaT,SEA,W BOUND,1805P,SEATTLE TRACON,"KEFLAVIK, ICELAND (BIKF) - SEA",...,433.0,NaN,US,US-WA,Seattle,KSEA,SEA,KSEA,SEA,"47.447943, -122.310276"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7577,NaN,FLORIDA,WEST PALM BEACH,"PRELIM INFO FROM FAA OPS: WEST PALM BEACH, FL/...",NaT,DFW,E BOUND,1800E,PALM BEACH TRACON,DFW - PBI,...,607.0,NaN,US,US-TX,Dallas-Fort Worth,KDFW,DFW,KDFW,DFW,"32.896801, -97.038002"
7578,NaN,FLORIDA,FORT MYERS,"PRELIM INFO FROM FAA OPS: FORT MYERS, FL/UAS I...",NaT,APF,S BOUND,1655E,FORT MYERS APCH,JVY - APF,...,8.0,NaN,US,US-FL,Naples,KAPF,APF,KAPF,APF,"26.1525993347, -81.7752990723"
7579,NaN,FLORIDA,NAPLES,_x000D_\nSummary: A/C WAS SOUTHBOUND TO APF AT...,NaT,APF,NaN,NaN,NaN,NaN,...,8.0,NaN,US,US-FL,Naples,KAPF,APF,KAPF,APF,"26.1525993347, -81.7752990723"
7580,NaN,HAWAII,HONOLULU,"PRELIM INFO FROM FAA OPS: HONOLULU, HI/UAS INC...",NaT,HNL,NaN,1512L,NaN,"HNL - LAHAINA, HI (PHJH)",...,13.0,NaN,US,US-HI,"Honolulu, Oahu",PHNL,HNL,PHNL,HNL,"21.32062, -157.924228"


In [19]:
merged_works_df_usa.to_csv("USA_merged_works.csv")

In [23]:
merged_works_df.columns

Index(['Day of Sighting', 'State', 'City', 'Summary', 'Date', 'code',
       'Direction', 'Time', 'Authority Advised', 'Route', 'Location',
       'Altitude', 'ident', 'type', 'name', 'elevation_ft', 'continent',
       'iso_country', 'iso_region', 'municipality', 'icao_code', 'iata_code',
       'gps_code', 'local_code', 'coordinates'],
      dtype='object')

In [24]:
import numpy as np
import pandas as pd

# Step 1: Split 'coordinates' column into 'latitude' and 'longitude'
merged_works_df[['latitude', 'longitude']] = merged_works_df['coordinates'].str.split(',', expand=True)
merged_works_df['latitude'] = pd.to_numeric(merged_works_df['latitude'].str.strip(), errors='coerce')
merged_works_df['longitude'] = pd.to_numeric(merged_works_df['longitude'].str.strip(), errors='coerce')

# Step 2: Drop rows with missing lat/long
merged_works_df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Step 3: (Optional) If no altitude column, assume 0 meters
merged_works_df['alt_meters'] = 0.0  # You can replace with actual altitude if available

# Step 4: Convert to 3D Cartesian coordinates
axis = 6378137
eccentricity = 8.1819190842622e-2
asq = np.power(axis, 2)
esq = np.power(eccentricity, 2)

def convert(df):
    df['lat_radian'] = np.radians(df['latitude'])
    df['long_radian'] = np.radians(df['longitude'])

    df['N'] = axis / np.sqrt(1 - esq * np.power(np.sin(df['lat_radian']), 2))

    df['x'] = (df['N'] + df['alt_meters']) * np.cos(df['lat_radian']) * np.cos(df['long_radian'])
    df['y'] = (df['N'] + df['alt_meters']) * np.cos(df['lat_radian']) * np.sin(df['long_radian'])
    df['z'] = ((1 - esq) * df['N'] + df['alt_meters']) * np.sin(df['lat_radian'])

    return df.drop(columns=['lat_radian', 'long_radian', 'N'])

# Step 5: Apply conversion
merged_works_df = convert(merged_works_df)

# Preview result
merged_works_df[['x', 'y', 'z']].head()

,x,y,z
0,8.510401e+05,-4.792341e+06,4.108053e+06
1,9.694027e+05,-5.658393e+06,2.769798e+06
2,-2.305542e+06,-3.646725e+06,4.681750e+06
3,1.851842e+05,-5.490063e+06,3.230359e+06
4,-2.309663e+06,-3.652075e+06,4.675586e+06


In [26]:
merged_works_df.columns

Index(['Day of Sighting', 'State', 'City', 'Summary', 'Date', 'code',
       'Direction', 'Time', 'Authority Advised', 'Route', 'Location',
       'Altitude', 'ident', 'type', 'name', 'elevation_ft', 'continent',
       'iso_country', 'iso_region', 'municipality', 'icao_code', 'iata_code',
       'gps_code', 'local_code', 'coordinates', 'latitude', 'longitude',
       'alt_meters', 'x', 'y', 'z'],
      dtype='object')

In [28]:
merged_works_df = merged_works_df[['Day of Sighting', 'x', 'y', 'z', 'State', 'City', 'Date', 'code',
       'Direction', 'Time', 'Authority Advised', 'Route', 'Location',
       'Altitude', 'ident', 'type', 'elevation_ft', 'continent',
       'iso_country', 'iso_region', 'municipality', 'icao_code', 'iata_code',
       'gps_code', 'local_code', 'coordinates', 'latitude', 'longitude',
       'alt_meters']]

In [30]:
merged_works_df.to_csv("xyzdataframe_small.csv")

In [33]:
merged_works_df

,Day of Sighting,x,y,z,State,City,Date,code,Direction,Time,...,iso_region,municipality,icao_code,iata_code,gps_code,local_code,coordinates,latitude,longitude,alt_meters
0,4/1/2022,8.510401e+05,-4.792341e+06,4.108053e+06,PENNSYLVANIA,PITTSBURGH,NaT,AGC,S BOUND,2358E,...,US-PA,Pittsburgh,KAGC,AGC,KAGC,AGC,"40.354401, -79.930199",40.354401,-79.930199,0.0
1,4/1/2022,9.694027e+05,-5.658393e+06,2.769798e+06,FLORIDA,MIAMI,NaT,OPF,W BOUND,1205E,...,US-FL,Miami,KOPF,OPF,KOPF,OPF,"25.907, -80.278397",25.907000,-80.278397,0.0
2,4/1/2022,-2.305542e+06,-3.646725e+06,4.681750e+06,WASHINGTON,SEATTLE,NaT,BFI,NNE BOUND,1210P,...,US-WA,Seattle,KBFI,BFI,KBFI,BFI,"47.529999, -122.302002",47.529999,-122.302002,0.0
3,4/1/2022,1.851842e+05,-5.490063e+06,3.230359e+06,ALABAMA,MOBILE,NaT,BFM,SE BOUND,1859C,...,US-AL,Mobile,KBFM,BFM,KBFM,BFM,"30.626800537100003, -88.06809997559999",30.626801,-88.068100,0.0
4,4/1/2022,-2.309663e+06,-3.652075e+06,4.675586e+06,WASHINGTON,SEATTLE,NaT,SEA,W BOUND,1805P,...,US-WA,Seattle,KSEA,SEA,KSEA,SEA,"47.447943, -122.310276",47.447943,-122.310276,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7577,NaN,-6.568338e+05,-5.320307e+06,3.444354e+06,FLORIDA,WEST PALM BEACH,NaT,DFW,E BOUND,1800E,...,US-TX,Dallas-Fort Worth,KDFW,DFW,KDFW,DFW,"32.896801, -97.038002",32.896801,-97.038002,0.0
7578,NaN,8.195501e+05,-5.669967e+06,2.794247e+06,FLORIDA,FORT MYERS,NaT,APF,S BOUND,1655E,...,US-FL,Naples,KAPF,APF,KAPF,APF,"26.1525993347, -81.7752990723",26.152599,-81.775299,0.0
7579,NaN,8.195501e+05,-5.669967e+06,2.794247e+06,FLORIDA,NAPLES,NaT,APF,NaN,NaN,...,US-FL,Naples,KAPF,APF,KAPF,APF,"26.1525993347, -81.7752990723",26.152599,-81.775299,0.0
7580,NaN,-5.508464e+06,-2.234042e+06,2.304500e+06,HAWAII,HONOLULU,NaT,HNL,NaN,1512L,...,US-HI,"Honolulu, Oahu",PHNL,HNL,PHNL,HNL,"21.32062, -157.924228",21.320620,-157.924228,0.0


In [34]:
# Count UAS sightings per IATA airport code
top_airports = (
    merged_works_df.groupby('iata_code')
    .size()
    .reset_index(name='uas_sighting_count')
    .sort_values(by='uas_sighting_count', ascending=False)
)

# Show top 10 airports
top_airports.head(10)


,iata_code,uas_sighting_count
25,AND,431
576,SAN,284
350,LGA,238
484,ORD,221
171,DEN,208
43,ATL,204
469,OFF,186
173,DFW,176
341,LAX,171
386,MCO,147


In [38]:
merged_works_df.to_csv("big_dataset_for_OLS.csv")